XRAY DATASET

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [9]:
data_dir = 'D:\Work\web_project\Covid19_Dataset\Xray_dataset'

In [6]:
# Define transformations for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Resize to 224x224 (ResNet input size)
        transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
        transforms.ToTensor(),              # Convert images to PyTorch tensors
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),             # Resize to 256x256
        transforms.CenterCrop(224),         # Crop center to 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [10]:
# Load the full dataset
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Calculate sizes for train and validation splits
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation

# Split the dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply validation-specific transformations to the validation dataset
val_dataset.dataset.transform = data_transforms['val']

# Create dataloaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
}

# Get dataset sizes and class names
dataset_sizes = {
    'train': train_size,
    'val': val_size
}
class_names = full_dataset.classes

print(f"Class Names: {class_names}")
print(f"Dataset Sizes: {dataset_sizes}")

Class Names: ['Negative', 'Positive']
Dataset Sizes: {'train': 2742, 'val': 686}


In [11]:
# Load pretrained ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes: positive and negative

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

c:\Users\Yaseen\AppData\Local\anaconda3\envs\yolo\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Yaseen\AppData\Local\anaconda3\envs\yolo\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has the best accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    print(f'Best Validation Accuracy: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train Loss: 0.1072 Acc: 0.9690
val Loss: 9.6566 Acc: 0.6166

Epoch 2/10
----------
train Loss: 0.0531 Acc: 0.9818
val Loss: 0.0702 Acc: 0.9883

Epoch 3/10
----------
train Loss: 0.0300 Acc: 0.9902
val Loss: 0.0954 Acc: 0.9898

Epoch 4/10
----------
train Loss: 0.0303 Acc: 0.9905
val Loss: 0.0572 Acc: 0.9898

Epoch 5/10
----------
train Loss: 0.0333 Acc: 0.9920
val Loss: 0.0395 Acc: 0.9898

Epoch 6/10
----------
train Loss: 0.0178 Acc: 0.9942
val Loss: 0.0272 Acc: 0.9913

Epoch 7/10
----------
train Loss: 0.0103 Acc: 0.9971
val Loss: 0.0402 Acc: 0.9898

Epoch 8/10
----------
train Loss: 0.0128 Acc: 0.9971
val Loss: 0.0574 Acc: 0.9898

Epoch 9/10
----------
train Loss: 0.0174 Acc: 0.9956
val Loss: 0.1063 Acc: 0.9869

Epoch 10/10
----------
train Loss: 0.0146 Acc: 0.9953
val Loss: 0.0316 Acc: 0.9898

Best Validation Accuracy: 0.9913


In [14]:
# Save the model
torch.save(model.state_dict(), 'resnet50_covid19.pth')
print("Model saved successfully!")

Model saved successfully!


#Test on CT dataset

In [18]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the path to the test dataset
test_dir = 'D:\Work\web_project\Covid19_Dataset\Test data'

# Define transformations for the test dataset (same as validation)
test_transforms = transforms.Compose([
    transforms.Resize(256),             # Resize to 256x256
    transforms.CenterCrop(224),         # Crop center to 224x224
    transforms.ToTensor(),              # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
])

# Load the test dataset
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Create a dataloader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load the trained model
model_path = 'resnet50_covid19.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the ResNet50 model
model = models.resnet50(pretrained=False)  # No need to load pretrained weights again
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # 2 output classes: positive and negative
model.load_state_dict(torch.load(model_path))  # Load the saved weights
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to calculate test accuracy
def calculate_test_accuracy(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Update counts
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Calculate test accuracy
test_accuracy = calculate_test_accuracy(model, test_dataloader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

c:\Users\Yaseen\AppData\Local\anaconda3\envs\yolo\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Accuracy: 55.56%


#Fine tune on CT Dataset

In [20]:
new_data_dir = 'D:\Work\web_project\Covid19_Dataset/New_dataset'

In [21]:
# Define transformations for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Resize to 224x224 (ResNet input size)
        transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
        transforms.ToTensor(),              # Convert images to PyTorch tensors
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),             # Resize to 256x256
        transforms.CenterCrop(224),         # Crop center to 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [22]:
# Load the full dataset
full_dataset = datasets.ImageFolder(new_data_dir, transform=data_transforms['train'])

# Calculate sizes for train and validation splits
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation

# Split the dataset
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Apply validation-specific transformations to the validation dataset
val_dataset.dataset.transform = data_transforms['val']

# Create dataloaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
}

# Get dataset sizes and class names
dataset_sizes = {
    'train': train_size,
    'val': val_size
}
class_names = full_dataset.classes

print(f"Class Names: {class_names}")
print(f"Dataset Sizes: {dataset_sizes}")

Class Names: ['negative', 'positive']
Dataset Sizes: {'train': 1608, 'val': 402}


In [ ]:
# Load pretrained ResNet50 model
model = models.resnet50(pretrained=False)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes: positive and negative

# Load the saved weights from the previous model
model_path = 'resnet50_covid19.pth'
model.load_state_dict(torch.load(model_path))

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [23]:
# Load pretrained ResNet50 model
model = models.resnet50(pretrained=False)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes: positive and negative

# Load the saved weights from the previous model
model_path = 'resnet50_covid19.pth'
model.load_state_dict(torch.load(model_path))

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [24]:
# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the final fully connected layer
model.fc.requires_grad_(True)

# Verify which layers are trainable
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

conv1.weight: requires_grad=False
bn1.weight: requires_grad=False
bn1.bias: requires_grad=False
layer1.0.conv1.weight: requires_grad=False
layer1.0.bn1.weight: requires_grad=False
layer1.0.bn1.bias: requires_grad=False
layer1.0.conv2.weight: requires_grad=False
layer1.0.bn2.weight: requires_grad=False
layer1.0.bn2.bias: requires_grad=False
layer1.0.conv3.weight: requires_grad=False
layer1.0.bn3.weight: requires_grad=False
layer1.0.bn3.bias: requires_grad=False
layer1.0.downsample.0.weight: requires_grad=False
layer1.0.downsample.1.weight: requires_grad=False
layer1.0.downsample.1.bias: requires_grad=False
layer1.1.conv1.weight: requires_grad=False
layer1.1.bn1.weight: requires_grad=False
layer1.1.bn1.bias: requires_grad=False
layer1.1.conv2.weight: requires_grad=False
layer1.1.bn2.weight: requires_grad=False
layer1.1.bn2.bias: requires_grad=False
layer1.1.conv3.weight: requires_grad=False
layer1.1.bn3.weight: requires_grad=False
layer1.1.bn3.bias: requires_grad=False
layer1.2.conv1.wei

In [25]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Only optimize the parameters of the final fully connected layer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [26]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has the best accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    print(f'Best Validation Accuracy: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Fine-tune the model
model = train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train Loss: 0.7690 Acc: 0.6835
val Loss: 0.5455 Acc: 0.7388

Epoch 2/10
----------
train Loss: 0.5738 Acc: 0.6915
val Loss: 0.5168 Acc: 0.7488

Epoch 3/10
----------
train Loss: 0.5660 Acc: 0.7058
val Loss: 0.5135 Acc: 0.7189

Epoch 4/10
----------
train Loss: 0.5450 Acc: 0.7108
val Loss: 0.5395 Acc: 0.6866

Epoch 5/10
----------
train Loss: 0.5621 Acc: 0.7046
val Loss: 0.4968 Acc: 0.7811

Epoch 6/10
----------
train Loss: 0.5061 Acc: 0.7544
val Loss: 0.4723 Acc: 0.7687

Epoch 7/10
----------
train Loss: 0.5031 Acc: 0.7450
val Loss: 0.4599 Acc: 0.8159

Epoch 8/10
----------
train Loss: 0.4813 Acc: 0.7649
val Loss: 0.4857 Acc: 0.7289

Epoch 9/10
----------
train Loss: 0.4727 Acc: 0.7587
val Loss: 0.4478 Acc: 0.8035

Epoch 10/10
----------
train Loss: 0.4970 Acc: 0.7618
val Loss: 0.4571 Acc: 0.7935

Best Validation Accuracy: 0.8159


In [27]:
# Save the fine-tuned model
torch.save(model.state_dict(), 'resnet50_finetuned.pth')
print("Fine-tuned model saved successfully!")

Fine-tuned model saved successfully!


Test on CT dataset

In [30]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the path to the test dataset
test_dir = 'D:\Work\web_project\Covid19_Dataset\Test data'

# Define transformations for the test dataset (same as validation)
test_transforms = transforms.Compose([
    transforms.Resize(256),             # Resize to 256x256
    transforms.CenterCrop(224),         # Crop center to 224x224
    transforms.ToTensor(),              # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
])

# Load the test dataset
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Create a dataloader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load the trained model
model_path = 'resnet50_finetuned.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the ResNet50 model
model = models.resnet50(pretrained=False)  # No need to load pretrained weights again
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # 2 output classes: positive and negative
model.load_state_dict(torch.load(model_path))  # Load the saved weights
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to calculate test accuracy
def calculate_test_accuracy(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Update counts
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Calculate test accuracy
test_accuracy = calculate_test_accuracy(model, test_dataloader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 61.11%
